In [42]:
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
import tqdm
import copy
from model.model import get_3Dlandmark_reg_model
from data_load.pose_estim_dataset import load_data



class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [80]:
train_loader = load_data(['../Data_preparation/DatasetN2_3.csv'], model_type='M2', full_dataset=True)


In [81]:
def weigthed_mse_loss(predicted_pos, target_pos, visibility):
    mse_loss = torch.mean(visibility.unsqueeze(2)*(predicted_pos - target_pos)**2)
    return mse_loss

criterion = weigthed_mse_loss

In [75]:

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device.')

n_landmarks = 9

model = get_3Dlandmark_reg_model(n_landmarks=n_landmarks, is_train=True, training_path='versions/model1_M2_18_9.pt', device=device)


optimizer = optim.Adam(model.parameters(), lr=0.005)

epoch = 0
num_epochs = 150
done = False
es = EarlyStopping(patience=10)

while epoch < num_epochs and not done:
    epoch += 1
    model.train()
    
    steps = list(enumerate(train_loader))
    pbar = tqdm.tqdm(steps)

    for batch_index, (input, target, visibility) in pbar:
        input = input.to(device)
        target = target.to(device)
        visibility = visibility.to(device)

        output = model(input)
        
        loss = criterion(output, target, visibility)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (batch_index+1)*len(input)

        if batch_index == len(steps) - 1:
            model.eval()
            output = model(input)
            vloss = criterion(output, target, visibility)
            if es(model, vloss):
                done = True
            pbar.set_description(
                f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, {es.status}"
            )
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")
    


Using cpu device.
Pre-trained model loaded with versions/model1_M2_18_9.pt


Epoch: 1, tloss: 0.2554798722267151, vloss: 0.248375, : 100%|██████████| 113/113 [00:00<00:00, 151.14it/s]
Epoch: 2, tloss: 0.20888292789459229, vloss: 0.217625, Improvement found, counter reset to 0: 100%|██████████| 113/113 [00:00<00:00, 119.85it/s]
Epoch: 3, tloss: 0.2654554545879364, vloss: 0.196167, Improvement found, counter reset to 0: 100%|██████████| 113/113 [00:00<00:00, 148.51it/s]
Epoch: 4, tloss: 0.2909855246543884, vloss: 0.308972, No improvement in the last 1 epochs: 100%|██████████| 113/113 [00:00<00:00, 142.27it/s]
Epoch: 5, tloss: 0.2634470760822296, vloss: 0.260773, No improvement in the last 2 epochs: 100%|██████████| 113/113 [00:00<00:00, 136.93it/s]
Epoch: 6, tloss: 0.31587034463882446, vloss: 0.307754, No improvement in the last 3 epochs: 100%|██████████| 113/113 [00:00<00:00, 114.89it/s]
Epoch: 7, tloss: 0.4155537188053131, vloss: 0.366240, No improvement in the last 4 epochs: 100%|██████████| 113/113 [00:00<00:00, 119.00it/s]
Epoch: 8, tloss: 0.3497952520847320

In [77]:
#n_landmarks = 9
#device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = get_3Dlandmark_reg_model(n_landmarks=n_landmarks, is_train=True, training_path='versions/model1_M2_18_9.pt', device=device)

Pre-trained model loaded with versions/model1_M2_18_9.pt


In [78]:
val_loader = load_data(['../Data_preparation/DatasetN3_3.csv'], model_type='M2', full_dataset=True)

def error_analysis(target, output, visibility):
    error = visibility.unsqueeze(2)*torch.abs(target - output)
    error = torch.mean(error, axis=0)
    error = torch.mean(error, axis=0)

    return error

# Evaluation on validation set
model.eval()
val_loss = 0.0
error_vect = []
with torch.no_grad():
    for input, target, visibility in val_loader:
        input = input.to(device)
        target = target.to(device)
        visibility = visibility.to(device)
        output = model(input)
        error_vect.append(error_analysis(target, output, visibility))
        current = criterion(output, target, visibility).item()
        val_loss += current

print(f'Validation Loss: {val_loss / len(val_loader)}')
error = np.mean(np.array(error_vect), axis=0)
print(f'Average error: {error}')


Validation Loss: 0.8795979086841855
Average error: [0.41147593 0.3897819  0.8167491 ]


In [79]:
import math
# Prediction on test set
model.eval()
predictions = []
ground_truth = []
in_values = []
dict = {}
for w in range(20, 3, -1):
    dict[w] = {'mean': 0, 'error': 0, 'count': 0, 'v_count': 0}
with torch.no_grad():
    for input, target, visibility in val_loader:
        input = input.to(device)
        target = target.to(device)
        visibility = visibility.to(device)
        output = model(input)

        input = input.cpu().numpy()
        target = target.cpu().numpy()
        visibility = visibility.cpu().numpy()
        output = output.cpu().numpy()

        for i in range(output.shape[0]):
            in_frame = target[i,np.where(visibility[i] == 1),:].squeeze(0)
            in_frame_out = output[i,np.where(visibility[i] == 1),:].squeeze(0)
            v_range = math.floor(np.abs(np.mean(in_frame[:,-1], axis=0))/10)
            if v_range in dict.keys():
                dict[v_range]['error'] += np.mean(np.abs(in_frame - in_frame_out), axis=0)
                dict[v_range]['count'] += 1
                dict[v_range]['v_count'] += np.sum(visibility[i,:])/visibility.shape[1]
                
        in_values.append(input)
        predictions.append(output)
        ground_truth.append(target)
print(f'Prediction: {dict}')
for k in dict.keys():
    if dict[k]['count'] != 0:
        dict[k]['mean'] = np.round(dict[k]['error']/dict[k]['count'],2)
        dict[k]['v_count'] = np.round(dict[k]['v_count']/dict[k]['count'],2)
    print(f'Range: {k*10}-{(k+1)*10} cm, Mean error: ({dict[k]["mean"]})cm --> number of samples: {dict[k]["count"]}, visibility: {dict[k]["v_count"]}')



Prediction: {20: {'mean': 0, 'error': 0, 'count': 0, 'v_count': 0}, 19: {'mean': 0, 'error': 0, 'count': 0, 'v_count': 0}, 18: {'mean': 0, 'error': 0, 'count': 0, 'v_count': 0}, 17: {'mean': 0, 'error': 0, 'count': 0, 'v_count': 0}, 16: {'mean': 0, 'error': array([ 3.7787595,  1.301108 , 11.384624 ], dtype=float32), 'count': 12, 'v_count': 12.0}, 15: {'mean': 0, 'error': array([ 55.84934,  46.60326, 144.23085], dtype=float32), 'count': 166, 'v_count': 166.0}, 14: {'mean': 0, 'error': array([ 66.450195,  87.52027 , 176.31114 ], dtype=float32), 'count': 206, 'v_count': 206.0}, 13: {'mean': 0, 'error': array([ 84.255974, 101.34584 , 179.60997 ], dtype=float32), 'count': 206, 'v_count': 206.0}, 12: {'mean': 0, 'error': array([ 94.18993, 119.08504, 165.84067], dtype=float32), 'count': 204, 'v_count': 204.0}, 11: {'mean': 0, 'error': array([111.818016,  96.846466, 178.46942 ], dtype=float32), 'count': 216, 'v_count': 211.3333333333332}, 10: {'mean': 0, 'error': array([ 94.346954,  70.46107 ,

In [ ]:
torch.save(model.state_dict(), 'versions/model3_M3_18_9.pt')

In [ ]:
import sys
sys.path.append('../')
from CAD_transf import get_points
ground_truth = np.array(get_points())*100
ground_truth[:,2] -= ground_truth[0,2]
np.savetxt('CPD/data/ground_truth.txt', ground_truth, delimiter=' ')

data = pd.read_csv('../Data_preparation/DatasetN2_3.csv')
points3D = []
points2D = []
for i in range(9):
    points2D.append(f'LDM{i}x')
    points2D.append(f'LDM{i}y')
    points3D.append(f'x{i}')
    points3D.append(f'y{i}')
    points3D.append(f'z{i}')

points2D = np.array(data[points2D].values[0]).reshape(1,18)/512
points3D = np.array(data[points3D].values[0]).reshape(1,9,3)

print('Image orientation: ', (data['roll'].values[0], data['pitch'].values[0], data['yaw'].values[0]))
print('Camera position: ', (data['x'].values[0], data['y'].values[0], data['z'].values[0]))

#print(points2D)
print(points3D)

out = model(torch.tensor(points2D, dtype=torch.float32).to(device))
out = out.squeeze(0).detach().numpy()
print(out)
np.savetxt('CPD/data/prediction.txt', out, delimiter=' ') 




In [ ]:
torch.save(model.state_dict(), 'versions/model1_18_9.pt')

(X,Y,Z) from perspective matrix and Z